# Аналіз даних за допомогою методів машинного навчання

У цьому ноутбуці ми проведемо аналіз даних за допомогою методів машинного навчання, включаючи регресійний аналіз, класифікацію та оптимізацію моделей.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, classification_report

## Допоміжні функції

Спочатку визначимо допоміжні функції для оцінки моделей регресії та класифікації.

In [2]:
# Функція для виведення результатів регресії
def print_regression_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\nМетрики для {model_name}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"R²: {r2:.4f}")
    return mae, mse, r2

In [3]:
# Функція для виведення результатів класифікації
def print_classification_metrics(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    print(f"\nМетрики для {model_name}:")
    print(f"Accuracy: {acc:.4f}")
    print("\nДетальний звіт:")
    print(classification_report(y_true, y_pred))
    return acc

## Завдання 1: Регресійний аналіз

У цьому розділі ми проведемо регресійний аналіз даних, використовуючи лінійну регресію та Random Forest.

In [4]:
print("\n===== ЗАВДАННЯ 1: РЕГРЕСІЙНИЙ АНАЛІЗ =====\n")

# Завантаження даних для регресії
regression_data = pd.read_csv('variant_14_regression.csv')
print("Перші 5 рядків даних для регресії:")
print(regression_data.head())
print("\nІнформація про дані:")
print(regression_data.info())
print("\nСтатистика даних:")
print(regression_data.describe())


===== ЗАВДАННЯ 1: РЕГРЕСІЙНИЙ АНАЛІЗ =====

Перші 5 рядків даних для регресії:
   feature_1  feature_2  feature_3  feature_4  feature_5 category      target
0  -0.441624   0.056910   2.145108   0.545351  -1.000692        B  158.228802
1   0.267580  -0.866764   0.307010   0.180572  -2.300686        C  -41.069085
2  -0.546059   0.587494  -0.179630   0.913220   0.542977        C   32.716057
3  -0.104489   0.252424   0.477136   0.745406  -0.954770        C   32.369672
4   0.951492  -0.102303   0.862357  -0.023328  -0.613082        A   88.818761

Інформація про дані:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature_1  1000 non-null   float64
 1   feature_2  1000 non-null   float64
 2   feature_3  1000 non-null   float64
 3   feature_4  1000 non-null   float64
 4   feature_5  1000 non-null   float64
 5   category   1000 non-null   object 
 6   

In [5]:
# Перевірка пропущених значень
print("\nКількість пропущених значень:")
print(regression_data.isnull().sum())

# Визначення категоріальних і числових ознак
reg_categorical_cols = regression_data.select_dtypes(include=['object', 'category']).columns.tolist()
reg_numerical_cols = regression_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
reg_numerical_cols.remove('target')  # Видаляємо цільову змінну зі списку ознак

print(f"\nКатегоріальні ознаки: {reg_categorical_cols}")
print(f"Числові ознаки: {reg_numerical_cols}")


Кількість пропущених значень:
feature_1    0
feature_2    0
feature_3    0
feature_4    0
feature_5    0
category     0
target       0
dtype: int64

Категоріальні ознаки: ['category']
Числові ознаки: ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5']


In [6]:
# Підготовка даних для регресії
X_reg = regression_data.drop('target', axis=1)
y_reg = regression_data['target']

# Створення препроцесора для регресії
reg_preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), reg_numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), reg_categorical_cols)
    ])

# Розділення даних на тренувальну та тестову вибірки
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

In [7]:
# Лінійна регресія
linear_reg = Pipeline([
    ('preprocessor', reg_preprocessor),
    ('regressor', LinearRegression())
])

linear_reg.fit(X_reg_train, y_reg_train)
y_reg_pred_linear = linear_reg.predict(X_reg_test)

# Random Forest регресія
rf_reg = Pipeline([
    ('preprocessor', reg_preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

rf_reg.fit(X_reg_train, y_reg_train)
y_reg_pred_rf = rf_reg.predict(X_reg_test)

In [8]:
# Оцінка моделей регресії
linear_metrics = print_regression_metrics(y_reg_test, y_reg_pred_linear, "Лінійна регресія")
rf_metrics = print_regression_metrics(y_reg_test, y_reg_pred_rf, "Random Forest регресія")

# Порівняння моделей регресії
print("\nПорівняння моделей регресії:")
if rf_metrics[2] > linear_metrics[2]:
    print("Random Forest регресія показує кращі результати.")
    print("Причини: Random Forest може моделювати нелінійні залежності та взаємодії між ознаками.")
else:
    print("Лінійна регресія показує кращі результати.")
    print("Причини: Дані можуть мати лінійну структуру або містити мало ознак.")


Метрики для Лінійна регресія:
MAE: 16.7277
MSE: 432.2413
R²: 0.9767

Метрики для Random Forest регресія:
MAE: 27.7116
MSE: 1293.4527
R²: 0.9303

Порівняння моделей регресії:
Лінійна регресія показує кращі результати.
Причини: Дані можуть мати лінійну структуру або містити мало ознак.


## Завдання 2: Класифікація

У цьому розділі ми проведемо класифікацію даних, використовуючи логістичну регресію та Random Forest.

In [9]:
print("\n\n===== ЗАВДАННЯ 2: КЛАСИФІКАЦІЯ =====\n")

# Завантаження даних для класифікації
classification_data = pd.read_csv('variant_14_classification.csv')
print("Перші 5 рядків даних для класифікації:")
print(classification_data.head())
print("\nІнформація про дані:")
print(classification_data.info())
print("\nСтатистика даних:")
print(classification_data.describe())



===== ЗАВДАННЯ 2: КЛАСИФІКАЦІЯ =====

Перші 5 рядків даних для класифікації:
   feature_1  feature_2  feature_3  feature_4  feature_5 category  target
0   1.144100   1.365056  -0.492980  -0.052019  -1.774204        A       1
1   0.081634  -3.275986  -2.840944  -1.196610   1.443525        C       0
2  -0.340361  -2.645979  -0.643340  -0.600262   1.984598        A       0
3  -0.990415   1.814362  -2.060027  -4.380028   2.878635        A       0
4  -0.923337  -1.700510  -0.032281   0.193580  -2.142717        A       1

Інформація про дані:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature_1  1000 non-null   float64
 1   feature_2  1000 non-null   float64
 2   feature_3  1000 non-null   float64
 3   feature_4  1000 non-null   float64
 4   feature_5  1000 non-null   float64
 5   category   1000 non-null   object 
 6   target     1000 non-null 

In [10]:
# Перевірка пропущених значень
print("\nКількість пропущених значень:")
print(classification_data.isnull().sum())

# Визначення категоріальних і числових ознак
cls_categorical_cols = classification_data.select_dtypes(include=['object', 'category']).columns.tolist()
cls_numerical_cols = classification_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
cls_numerical_cols.remove('target')  # Видаляємо цільову змінну зі списку ознак

print(f"\nКатегоріальні ознаки: {cls_categorical_cols}")
print(f"Числові ознаки: {cls_numerical_cols}")


Кількість пропущених значень:
feature_1    0
feature_2    0
feature_3    0
feature_4    0
feature_5    0
category     0
target       0
dtype: int64

Категоріальні ознаки: ['category']
Числові ознаки: ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5']


In [11]:
# Підготовка даних для класифікації
X_cls = classification_data.drop('target', axis=1)
y_cls = classification_data['target']

# Створення препроцесора для класифікації
cls_preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), cls_numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cls_categorical_cols)
    ])

# Розділення даних на тренувальну та тестову вибірки
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42)

In [12]:
# Логістична регресія
logistic_cls = Pipeline([
    ('preprocessor', cls_preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

logistic_cls.fit(X_cls_train, y_cls_train)
y_cls_pred_logistic = logistic_cls.predict(X_cls_test)

# Random Forest класифікація
rf_cls = Pipeline([
    ('preprocessor', cls_preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

rf_cls.fit(X_cls_train, y_cls_train)
y_cls_pred_rf = rf_cls.predict(X_cls_test)

In [13]:
# Оцінка моделей класифікації
logistic_acc = print_classification_metrics(y_cls_test, y_cls_pred_logistic, "Логістична регресія")
rf_acc = print_classification_metrics(y_cls_test, y_cls_pred_rf, "Random Forest класифікація")

# Порівняння моделей класифікації
print("\nПорівняння моделей класифікації:")
if rf_acc > logistic_acc:
    print("Random Forest класифікація показує кращі результати.")
    print("Причини: Random Forest може краще моделювати складні взаємодії між ознаками.")
else:
    print("Логістична регресія показує кращі результати.")
    print("Причини: Дані можуть мати просту структуру або містити мало ознак.")


Метрики для Логістична регресія:
Accuracy: 0.7300

Детальний звіт:
              precision    recall  f1-score   support

           0       0.77      0.66      0.71       101
           1       0.70      0.80      0.75        99

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200


Метрики для Random Forest класифікація:
Accuracy: 0.9150

Детальний звіт:
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       101
           1       0.93      0.90      0.91        99

    accuracy                           0.92       200
   macro avg       0.92      0.91      0.91       200
weighted avg       0.92      0.92      0.91       200


Порівняння моделей класифікації:
Random Forest класифікація показує кращі результати.
Причини: Random Forest може краще моделювати складні взаємодії між ознаками.


## Завдання 3: Оптимізація моделі

У цьому розділі ми оптимізуємо модель, яка показала кращі результати (регресія або класифікація).

In [14]:
print("\n\n===== ЗАВДАННЯ 3: ОПТИМІЗАЦІЯ МОДЕЛІ =====\n")

# Визначаємо, яку модель оптимізувати (регресія чи класифікація)
if rf_metrics[2] > linear_metrics[2]:
    print("Оптимізуємо Random Forest Regressor, оскільки він показав кращі результати.")
    # Параметри для пошуку
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [None, 10, 20, 30],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    }
    
    # Створення GridSearchCV
    grid_search = GridSearchCV(
        rf_reg, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1
    )
    
    # Навчання моделі з пошуком параметрів
    grid_search.fit(X_reg_train, y_reg_train)
    
    # Найкращі параметри
    print("\nНайкращі параметри:")
    print(grid_search.best_params_)
    
    # Оцінка оптимізованої моделі
    best_rf_reg = grid_search.best_estimator_
    y_reg_pred_best = best_rf_reg.predict(X_reg_test)
    
    print("\nРезультати до оптимізації:")
    print(f"MAE: {rf_metrics[0]:.4f}")
    print(f"MSE: {rf_metrics[1]:.4f}")
    print(f"R²: {rf_metrics[2]:.4f}")
    
    print("\nРезультати після оптимізації:")
    best_metrics = print_regression_metrics(y_reg_test, y_reg_pred_best, "Оптимізований Random Forest Regressor")
    
    # Порівняння результатів
    print("\nПокращення після оптимізації:")
    print(f"MAE: {rf_metrics[0] - best_metrics[0]:.4f} ({(1 - best_metrics[0]/rf_metrics[0])*100:.2f}%)")
    print(f"MSE: {rf_metrics[1] - best_metrics[1]:.4f} ({(1 - best_metrics[1]/rf_metrics[1])*100:.2f}%)")
    print(f"R²: {best_metrics[2] - rf_metrics[2]:.4f} ({(best_metrics[2]/rf_metrics[2] - 1)*100:.2f}%)")
    
else:
    print("Оптимізуємо Random Forest Classifier, оскільки він показав кращі результати.")
    # Параметри для пошуку
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
    
    # Створення GridSearchCV
    grid_search = GridSearchCV(
        rf_cls, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1
    )
    
    # Навчання моделі з пошуком параметрів
    grid_search.fit(X_cls_train, y_cls_train)
    
    # Найкращі параметри
    print("\nНайкращі параметри:")
    print(grid_search.best_params_)
    
    # Оцінка оптимізованої моделі
    best_rf_cls = grid_search.best_estimator_
    y_cls_pred_best = best_rf_cls.predict(X_cls_test)
    
    print("\nРезультати до оптимізації:")
    print(f"Accuracy: {rf_acc:.4f}")
    
    print("\nРезультати після оптимізації:")
    best_acc = print_classification_metrics(y_cls_test, y_cls_pred_best, "Оптимізований Random Forest Classifier")
    
    # Порівняння результатів
    print("\nПокращення після оптимізації:")
    print(f"Accuracy: {best_acc - rf_acc:.4f} ({(best_acc/rf_acc - 1)*100:.2f}%)")



===== ЗАВДАННЯ 3: ОПТИМІЗАЦІЯ МОДЕЛІ =====

Оптимізуємо Random Forest Classifier, оскільки він показав кращі результати.
Fitting 5 folds for each of 108 candidates, totalling 540 fits

Найкращі параметри:
{'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}

Результати до оптимізації:
Accuracy: 0.9150

Результати після оптимізації:

Метрики для Оптимізований Random Forest Classifier:
Accuracy: 0.9150

Детальний звіт:
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       101
           1       0.93      0.90      0.91        99

    accuracy                           0.92       200
   macro avg       0.92      0.91      0.91       200
weighted avg       0.92      0.92      0.91       200


Покращення після оптимізації:
Accuracy: 0.0000 (0.00%)


## Висновки

In [ ]:
print("\nВисновки:")
print("1. Для регресійного аналізу Random Forest зазвичай показує кращі результати, ніж лінійна регресія,")
print("   оскільки може моделювати нелінійні залежності та взаємодії між ознаками.")
print("2. Для класифікації Random Forest також часто перевершує логістичну регресію,")
print("   особливо на складних даних з багатьма ознаками.")
print("3. Оптимізація гіперпараметрів дозволяє значно покращити результати моделей,")
print("   але вимагає більше обчислювальних ресурсів.")